In [ ]:
!pip install fastf1
!pip install Pillow scikit-learn scipy matplotlib --quiet
import bokeh
import fastf1
import logging
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
import pandas as pd
import numpy as np
import os
os.makedirs('/content/fastf1_cache', exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.2 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [ ]:
import logging
import json
import os

# Enable caching
os.makedirs("cache", exist_ok=True)
fastf1.Cache.enable_cache('cache')

logging.getLogger("fastf1").setLevel(logging.WARNING)


session = fastf1.get_session(2024, 'Miami', 'R')  # Modify as needed
session.load()


driver = 'STR'  # You can change the driver
laps = session.laps.pick_drivers(driver).pick_fastest()


car_data = laps.get_car_data().add_distance()
df = car_data

# check if required columns are available (Speed, Throttle, Brake)
if 'Speed' not in df.columns or 'Throttle' not in df.columns or 'Brake' not in df.columns:
    print(f"Warning: Driver {driver} has missing telemetry data. Skipping...")
else:

    telemetry_data = []
    for i in range(len(df)):
        data_point = {
            "time": float(df['Time'].iloc[i].total_seconds()),
            "speed": float(df['Speed'].iloc[i]),
            "throttle": float(df['Throttle'].iloc[i]),
            "brake": float(df['Brake'].iloc[i])
        }
        telemetry_data.append(data_point)

    # Save the data to a JSON file
    with open("telemetry_STR_miami.json", "w") as f:
        json.dump(telemetry_data, f)

    print(f"Saved {len(telemetry_data)} data points for driver {driver}")


Saved 338 data points for driver STR


In [ ]:
from google.colab import files

# Save the JSON file
with open('telemetry_STR_miami.json', 'w') as f:
    json.dump(telemetry_data, f)

# Trigger download
files.download('telemetry_STR_miami.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import logging
import json
from google.colab import files

logging.getLogger("fastf1").setLevel(logging.WARNING)

os.makedirs('/content/fastf1_cache', exist_ok=True)
fastf1.Cache.enable_cache('/content/fastf1_cache')


session = fastf1.get_session(2024, 'Miami', 'Q')
session.load()

lap = session.laps.pick_drivers('NOR').pick_fastest()
tel = lap.get_telemetry()

# After getting telemetry
xs = tel['X'].to_numpy()
ys = tel['Y'].to_numpy()

# Canvas target dimensions
canvas_w = 879
canvas_h = 701

# Get original bounds
min_x, max_x = xs.min(), xs.max()
min_y, max_y = ys.min(), ys.max()

track_width = max_x - min_x
track_height = max_y - min_y

# Compute scale factor that preserves aspect ratio
scale = min(canvas_w / track_width, canvas_h / track_height)

# Center the track
offset_x = (canvas_w - track_width * scale) / 2
offset_y = (canvas_h - track_height * scale) / 2

# Normalize points with aspect ratio preserved
track_points = []
for x, y in zip(xs, ys):
    norm_x = (x - min_x) * scale + offset_x
    norm_y = (y - min_y) * scale + offset_y
    track_points.append({'x': round(norm_x, 1), 'y': round(norm_y, 1)})

# Save and download
with open('Miami_Track.json', 'w') as f:
    json.dump(track_points, f)

files.download('Miami_Track.json')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>